<a href="https://colab.research.google.com/github/xariskap/Computational-Intelligence/blob/main/Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np

In [2]:
idf = []
tf = []
tf_idf = []
bag_of_words = []

with open('/content/drive/MyDrive/Data/train-data.dat') as data:
    for line in data:
        tf_bag = [0 for _ in range(8520)]
        bag = [0 for _ in range(8520)]
        inline = line.strip().split()
        for index, value in enumerate(inline):
            if value[0] != '<':
                bag[int(value)] += 1
      
        bag_of_words.append(bag)
        
        # compute tf
        non_zero = np.count_nonzero(bag)
        for index, value in enumerate(bag):
          tf_bag[index] = value / non_zero
        tf.append(tf_bag)

np.array(bag_of_words)
N = np.size(bag_of_words,0)
M = np.size(bag_of_words,1)
sum_d_t = (np.count_nonzero(bag_of_words, axis=0))

# compute idf
for i in range(M):
    idf.append( np.log10( N / sum_d_t[i]) )

tf = np.array(tf)
idf = np.array(idf)

# cumpute tf-idf
tf_idf = tf * idf
tf_idf = np.array(tf_idf)


# tf_idf_mean
tf_idf_mean = np.mean(tf_idf, axis = 0)
max_fitness = (sum(tf_idf_mean))

In [118]:
class Individual(object):
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.fitness = self.calculate_fitness()

    @classmethod
    def create_genome(self):
        genome = np.random.randint(0, 1, size=8520)
        rand_ones = random.randint(0, 8519)
        for _ in range(rand_ones):
            index = random.randint(0, 8519)
            
            while genome[index] == 1:
                index = random.randint(0, 8519)
            
            genome[index] = 1
    
                
        return genome


    def repair_genome(self):

        non_zeros = np.sum(self.chromosome)
        
        while non_zeros < 1000:  
            difference = 1000 - non_zeros
            for _ in range(difference):
                self.chromosome[random.randint(0, 8519)] = 1

            non_zeros = np.sum(self.chromosome)
        
        return self.chromosome

    
    def calculate_fitness(self):
        global max_fitness
        global tf_idf_mean

        fitness = 0
        for index, gene in enumerate(self.chromosome):
            if gene == 1:
                fitness += tf_idf_mean[index]

        if np.count_nonzero(self.chromosome) > 2000:
            percentage = ((np.count_nonzero(self.chromosome) // 100) / 100)
            fitness -= percentage * max_fitness

        return fitness

In [110]:
def initialize_population(no_of_chromosomes):
    population = []
    for _ in range(no_of_chromosomes):
        genome = Individual.create_genome()
        genome = Individual(genome)
        
        #genome.genome_rejection()
        genome.repair_genome()
        
        population.append(genome)  # population (list) consists Individual objects 

    return np.array(population)

# def roulette_wheel_selection(population: list):
    
#     population_fitness = sum([chromosome.fitness for chromosome in population])
    
#     s = 0
#     random_number = random.randint(0, int(population_fitness))
    
#     for individual in population:
#         s += individual.fitness
#         if s >= random_number:
#             return individual

def tournament_selection(population: list):
    k = 5
    selected_parents = []
    for individual in range(k):
        index = random.randint(0, len(population) - 1)
        selected_parents.append(population[index])
    
    sorted_parents = sorted(selected_parents, key=lambda x: x.fitness, reverse = True)
    first_parent, second_parent = sorted_parents[0], sorted_parents[1]

    return first_parent, second_parent

def mate(papa: Individual, mama: Individual, p_cross):
    # 2-point crossover
    points = random.sample(range(len(mama.chromosome)), 2)
    points = sorted(points)

    papa_copy = papa.chromosome.copy()
    mama_copy = mama.chromosome.copy()

    if random.random() <= p_cross:
        papa.chromosome[0:points[0]] = mama_copy[0:points[0]]
        mama.chromosome[0:points[0]] = papa_copy[0:points[0]]
        papa.chromosome[points[1]:] = mama_copy[points[1]:]
        mama.chromosome[points[1]:] = papa_copy[points[1]:]

    first_child, second_child = papa, mama
    return first_child, second_child

def mutate(child: Individual, p_mutate):
    #if random.random() <= p_mutate:
    no_to_mutate = (5 * len(child.chromosome)) // 100
    for _ in range(no_to_mutate):
        random_index = random.randint(0, 8519)
        
        child.chromosome[random_index] = int(not child.chromosome[random_index])

    child.fitness = child.calculate_fitness()
    return child


In [123]:
def main():
    first_gen = initialize_population(100)
    
    #sorted_arr = sorted(first_gen, key=lambda x: x.fitness, reverse = True)
    #for individual in sorted_arr:
     #   print(f"{individual.fitness: <20}\t {np.count_nonzero(individual.chromosome)} genes")


    papa, mama = tournament_selection(first_gen)

    boy, girl  = mate(papa,mama,1)
    print(boy.fitness)
    print()

    

if __name__ == '__main__':
    main()


0.2395827053506489
0.2395827053506489
